
# Feature Engineering — Creación y enriquecimiento de variables
**Proyecto:** Análisis del desempeño logístico y satisfacción del cliente en ecommerce (México) <br>
**Versión:** v1




En este notebook se crean y documentan las variables derivadas que usaremos en el análisis exploratorio y los análisis posteriores.  
Aquí transformamos información temporal y operacional en indicadores claros (p. ej. días de entrega, retrasos, bandera de entrega tardía) y enriquecemos los pedidos con información sobre los costos y la experiencia del cliente, con el objetivo de contar con un dataset listo para el análisis exploratorio.


## Carga de datos

Se cargan los archivos que alimentarán las transformaciones: pedidos (orders), líneas de pedido (order_items), productos (products), reseñas (reviews) y vendedores (sellers).
Se trabajará sobre copias limpias (las generadas en la etapa anterior) para mantener trazabilidad y asegurar que las transformaciones sean reproducibles.

In [ ]:
# Importar librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Carga de datasets
base_path = "https://raw.githubusercontent.com/RaquelGlez/ecommerce_report/refs/heads/main/data/processed/"

orders = pd.read_csv(base_path + 'orders_clean.csv')
customers = pd.read_csv(base_path + 'customers_clean.csv')
order_items = pd.read_csv(base_path + 'order_items_clean.csv')
reviews = pd.read_csv(base_path + 'reviews_clean.csv')


# Vista general del dataset
print(f"Orders: {orders.shape} | Customers: {customers.shape}")
orders.head(2)




Orders: (3000, 8) | Customers: (3000, 5)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,ord_7370c26ead3d3cc6cee4f0548b8d,cus_3b0234601c0276e6cd4e08b6ae67,shipped,2025-08-16 21:12:27.948040,2025-08-16 23:12:27.948040,2025-08-17 23:12:27.948040,NaN,2025-08-24 23:12:27.948040
1,ord_079072dd7970d40a3922a6b0c459,cus_24cb3fa4569c4fd058f954f2e6f9,delivered,2024-12-12 02:52:21.349382,2024-12-12 05:52:21.349382,2024-12-14 05:52:21.349382,2024-12-15 05:52:21.349382,2024-12-15 05:52:21.349382


### Revisión inicial
Con la carga de datos lista se hace una inspección rápida para verificar: columnas presentes, tipos esperados, nulos evidentes y filas de muestra.
Esta revisión nos ayuda a decidir reglas de transformación (por ejemplo, qué fechas convertir y cómo tratar filas sin fecha de entrega).

### Conversión de tipos

Convertimos columnas temporales a datetime y normalizamos categorías. Usamos errors='coerce' para marcar con NaT cualquier valor no parseable y mantener trazabilidad.
Esto permite operar correctamente con diferencias entre fechas (por ejemplo, para calcular días de entrega).

In [ ]:
# Asegurar que las fechas estén en formato datetime
date_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

### Creación de variables temporales y de entrega
A partir de las fechas calculamos medidas que describen la experiencia de entrega:

- delivery_time_days: días reales desde la compra hasta la entrega.
- delay_vs_estimate: diferencia entre la entrega real y la fecha estimada (positivo = retraso).
- is_late: indicador binario que facilita agrupaciones rápidas.
- purchase_month: variable temporal para analizar estacionalidad.

Estas variables serán clave para relacionar desempeño operativo y satisfacción.

In [ ]:
# Calcular días entre compra y entrega, es decir duración real de entrega
orders["delivery_time_days"] = (
    orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]
).dt.days

In [ ]:
# Calcular diferencia entre fecha estimada y entrega real
orders["delay_vs_estimate"] = (
    orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"]
).dt.days

In [ ]:
# Agregar una bandera: 1 si el pedido se entregó después de la fecha estimada
orders["is_late"] = np.where(
    pd.isna(orders["delay_vs_estimate"]),  # condición 1: sin fecha de entrega
    None,                                  # valor si aún no se entregó
    np.where(orders["delay_vs_estimate"] > 0, 1, 0)  # valor si sí se entregó
)

In [ ]:
# Días y meses de compra (para análisis temporal)
orders["purchase_day"] = orders["order_purchase_timestamp"].dt.day_name()
orders["purchase_month"] = orders["order_purchase_timestamp"].dt.month_name()

In [ ]:
#Extraer el número de semana del año a partir de la fecha de compra (útil para detectar picos de ventas o entregas).
orders["week_of_year"] = orders["order_purchase_timestamp"].dt.isocalendar().week

In [ ]:
# Vista previa
orders[["order_id", "delivery_time_days", "delay_vs_estimate", "is_late", "purchase_day", "purchase_month", "week_of_year"]].head()

,order_id,delivery_time_days,delay_vs_estimate,is_late,purchase_day,purchase_month,week_of_year
0,ord_7370c26ead3d3cc6cee4f0548b8d,NaN,NaN,None,Saturday,August,33
1,ord_079072dd7970d40a3922a6b0c459,3.0,0.0,0,Thursday,December,50
2,ord_2600b1cb6f6a38610007a8c9142c,8.0,0.0,0,Friday,June,24
3,ord_aaa7c4626ea7ea636e5bba7afebd,2.0,-1.0,0,Saturday,April,17
4,ord_0ca0641b834ceb0a46f85637e125,4.0,-2.0,0,Saturday,June,25


In [ ]:
orders[["order_id", "delivery_time_days", "delay_vs_estimate", "is_late", "purchase_day", "purchase_month", "week_of_year"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            3000 non-null   object 
 1   delivery_time_days  2075 non-null   float64
 2   delay_vs_estimate   2075 non-null   float64
 3   is_late             2075 non-null   object 
 4   purchase_day        3000 non-null   object 
 5   purchase_month      3000 non-null   object 
 6   week_of_year        3000 non-null   UInt32 
dtypes: UInt32(1), float64(2), object(4)
memory usage: 155.4+ KB


Se observa que delivery_time_days tiene media ≈ 5.06 días (desviación ≈ 1.63) y que delay_vs_estimate tiene media ≈ -1.48 (lo que indica que en promedio las entregas tienden a llegar antes de lo prometido).
Podemos ver que delivery_time_days está presente en ~2,075 pedidos; el resto (≈925 pedidos) carece de fecha de entrega y por tanto mantiene NaT, esto refleja pedidos aún no entregados y se conservó intencionalmente.

Se agregan también dos variables clave al dataset `orders`

1. **`order_total_value`**→ representa el valor total del pedido, considerando el precio y el costo de envío (freight) de cada ítem.
2. **`review_score`** → refleja la satisfacción del cliente con el pedido, proveniente de las reseñas.

Estas variables nos permitirán realizar análisis posteriores sobre **el valor económico de los pedidos** y **la relación entre desempeño logístico y satisfacción del cliente**.


In [ ]:
print("orders:", orders.shape)
print("order_items:", order_items.shape)
print("reviews:", reviews.shape)

orders: (3000, 14)
order_items: (8904, 7)
reviews: (3000, 7)


In [ ]:
# Cálculo del valor total del pedido
# Cada pedido puede tener varios ítems, por lo tanto, debemos agrupar por order_id y sumar los precios y costos de envío (freight_value)

order_value = (
    order_items
    .groupby("order_id")[["price", "freight_value"]]
    .sum()
    .reset_index()
)

# Crear la nueva columna con el valor total del pedido
order_value["order_total_value"] = order_value["price"] + order_value["freight_value"]

# Verificación
order_value.head()


,order_id,price,freight_value,order_total_value
0,ord_000f6ab2d51f8a3fd4cd2a1f47cf,796.34,95.36,891.70
1,ord_00172c0b330712b10ed9fdd95624,9371.38,1616.42,10987.80
2,ord_00231f40212a3e2403e8b582f345,3092.78,709.43,3802.21
3,ord_0023c39a2e1f696b51170e405311,10916.91,2188.14,13105.05
4,ord_002467f1e91e994e9126fd281260,886.33,152.65,1038.98


In [ ]:
order_value.describe()

,price,freight_value,order_total_value
count,3000.000000,3000.000000,3000.000000
mean,7578.632947,1137.007203,8715.640150
std,4385.219800,721.157984,5052.011658
min,150.340000,16.890000,174.330000
25%,4054.045000,552.687500,4650.102500
50%,7169.550000,1050.345000,8268.690000
75%,10808.900000,1608.075000,12416.042500
max,21672.230000,4052.650000,25026.080000


In [ ]:
# Unión con el dataset principal (orders)

orders = orders.merge(order_value[["order_id", "order_total_value"]], on="order_id", how="left")

print("Dimensiones después del merge:", orders.shape)
orders.head()


Dimensiones después del merge: (3000, 15)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delay_vs_estimate,is_late,purchase_day,purchase_month,week_of_year,order_total_value
0,ord_7370c26ead3d3cc6cee4f0548b8d,cus_3b0234601c0276e6cd4e08b6ae67,shipped,2025-08-16 21:12:27.948040,2025-08-16 23:12:27.948040,2025-08-17 23:12:27.948040,NaT,2025-08-24 23:12:27.948040,NaN,NaN,None,Saturday,August,33,4949.98
1,ord_079072dd7970d40a3922a6b0c459,cus_24cb3fa4569c4fd058f954f2e6f9,delivered,2024-12-12 02:52:21.349382,2024-12-12 05:52:21.349382,2024-12-14 05:52:21.349382,2024-12-15 05:52:21.349382,2024-12-15 05:52:21.349382,3.0,0.0,0,Thursday,December,50,1276.09
2,ord_2600b1cb6f6a38610007a8c9142c,cus_67a2c2db5bd7dc2f06a8f9b40226,delivered,2025-06-13 03:04:55.053859,2025-06-13 19:04:55.053859,2025-06-16 19:04:55.053859,2025-06-21 19:04:55.053859,2025-06-21 19:04:55.053859,8.0,0.0,0,Friday,June,24,2137.68
3,ord_aaa7c4626ea7ea636e5bba7afebd,cus_2b70a9c550bfd5e065a756e9fe9b,delivered,2025-04-26 01:55:37.919760,2025-04-26 08:55:37.919760,2025-04-27 08:55:37.919760,2025-04-28 08:55:37.919760,2025-04-29 08:55:37.919760,2.0,-1.0,0,Saturday,April,17,989.99
4,ord_0ca0641b834ceb0a46f85637e125,cus_36ba5c7d572c22a6650f11e37f4d,delivered,2025-06-21 18:52:40.013157,2025-06-21 21:52:40.013157,2025-06-22 21:52:40.013157,2025-06-25 21:52:40.013157,2025-06-27 21:52:40.013157,4.0,-2.0,0,Saturday,June,25,4403.75


Después de tener `order_total_value` en el dataframe orders, en el notebook se observa que order_total_value presenta una media aproximada de 8,716 MXN y una dispersión significativa (std ≈ 5,052), lo que sugiere una variedad amplia en el ticket promedio.

### Incorporar calificaciones (review_score)

Traemos la columna review_score desde reviews y la agregamos por order_id. Así cada pedido queda asociado a su calificación (1–5).

In [ ]:
# Unión de calificaciones de clientes (review_score)

# Cada pedido tiene una reseña con un puntaje (1 a 5)
reviews_short = reviews[["order_id", "review_score"]]

# Unimos las calificaciones al dataset enriquecido
orders = orders.merge(reviews_short, on="order_id", how="left")

print("Dimensiones finales:", orders.shape)
orders.head()


Dimensiones finales: (3000, 16)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delay_vs_estimate,is_late,purchase_day,purchase_month,week_of_year,order_total_value,review_score
0,ord_7370c26ead3d3cc6cee4f0548b8d,cus_3b0234601c0276e6cd4e08b6ae67,shipped,2025-08-16 21:12:27.948040,2025-08-16 23:12:27.948040,2025-08-17 23:12:27.948040,NaT,2025-08-24 23:12:27.948040,NaN,NaN,None,Saturday,August,33,4949.98,3
1,ord_079072dd7970d40a3922a6b0c459,cus_24cb3fa4569c4fd058f954f2e6f9,delivered,2024-12-12 02:52:21.349382,2024-12-12 05:52:21.349382,2024-12-14 05:52:21.349382,2024-12-15 05:52:21.349382,2024-12-15 05:52:21.349382,3.0,0.0,0,Thursday,December,50,1276.09,4
2,ord_2600b1cb6f6a38610007a8c9142c,cus_67a2c2db5bd7dc2f06a8f9b40226,delivered,2025-06-13 03:04:55.053859,2025-06-13 19:04:55.053859,2025-06-16 19:04:55.053859,2025-06-21 19:04:55.053859,2025-06-21 19:04:55.053859,8.0,0.0,0,Friday,June,24,2137.68,4
3,ord_aaa7c4626ea7ea636e5bba7afebd,cus_2b70a9c550bfd5e065a756e9fe9b,delivered,2025-04-26 01:55:37.919760,2025-04-26 08:55:37.919760,2025-04-27 08:55:37.919760,2025-04-28 08:55:37.919760,2025-04-29 08:55:37.919760,2.0,-1.0,0,Saturday,April,17,989.99,3
4,ord_0ca0641b834ceb0a46f85637e125,cus_36ba5c7d572c22a6650f11e37f4d,delivered,2025-06-21 18:52:40.013157,2025-06-21 21:52:40.013157,2025-06-22 21:52:40.013157,2025-06-25 21:52:40.013157,2025-06-27 21:52:40.013157,4.0,-2.0,0,Saturday,June,25,4403.75,3


In [ ]:
orders.describe()

,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delay_vs_estimate,week_of_year,order_total_value,review_score
count,3000,2075,3000,2075.000000,2075.000000,3000.0,3000.000000,3000.000000
mean,2025-03-02 05:37:49.704598528,2025-03-06 02:55:47.922590976,2025-03-09 05:34:55.704598528,5.064578,-1.483855,26.381,8715.640150,3.804667
min,2024-09-01 23:43:28.753220,2024-09-05 04:23:43.069558,2024-09-07 04:23:43.069558,2.000000,-3.000000,1.0,174.330000,1.000000
25%,2024-12-01 16:38:55.337391104,2024-12-04 18:14:02.389638400,2024-12-08 17:17:50.851796480,4.000000,-2.000000,13.0,4650.102500,3.000000
50%,2025-02-27 19:37:01.748600576,2025-03-04 02:09:44.843072,2025-03-07 04:02:42.183006976,5.000000,-1.000000,27.0,8268.690000,4.000000
75%,2025-06-01 10:10:28.747568384,2025-06-03 23:57:40.305398016,2025-06-08 03:04:12.043620864,6.000000,0.000000,39.0,12416.042500,5.000000
max,2025-09-01 21:43:44.928373,2025-09-08 04:43:44.928373,2025-09-10 15:26:04.120227,9.000000,0.000000,52.0,25026.080000,5.000000
std,NaN,NaN,NaN,1.633979,1.113217,15.118511,5052.011658,1.181374


En el dataset final la calificación promedio es ≈ 3.80.
Los valores nulos en review_score indican pedidos sin reseña y se mantienen como información válida.

### Validaciones finales de las columnas agregadas

Revisamos cuántos pedidos carecen de order_total_value o review_score y analizamos un resumen estadístico de las nuevas columnas.

- order_total_value y review_score no presentan valores nulos significativos tras el merge.
- Las métricas temporales (delivery_time_days, delay_vs_estimate) tienen menos observaciones por la existencia de pedidos no entregados (caso válido).

Estos checks confirman que las uniones fueron exitosas y permiten avanzar al análisis exploratorio.

In [ ]:
# Validaciones básicas para columnas agregadas
# - Los valores nulos en `order_total_value` pueden deberse a pedidos sin ítems asociados.
# - Los valores nulos en `review_score` indican pedidos sin reseña del cliente.

# Comprobar cuántos pedidos no tienen valor total o calificación
missing_value_counts = orders[["order_total_value", "review_score"]].isnull().sum()
print(missing_value_counts)

# Estadísticos generales de las nuevas columnas
orders[["delivery_time_days", "delay_vs_estimate", "order_total_value", "review_score"]].describe()


order_total_value    0
review_score         0
dtype: int64


,delivery_time_days,delay_vs_estimate,order_total_value,review_score
count,2075.000000,2075.000000,3000.000000,3000.000000
mean,5.064578,-1.483855,8715.640150,3.804667
std,1.633979,1.113217,5052.011658,1.181374
min,2.000000,-3.000000,174.330000,1.000000
25%,4.000000,-2.000000,4650.102500,3.000000
50%,5.000000,-1.000000,8268.690000,4.000000
75%,6.000000,0.000000,12416.042500,5.000000
max,9.000000,0.000000,25026.080000,5.000000


### Resultado final del Feature Engineering

El dataset final ahora incluye:

| Categoría | Variables nuevas | Descripción |
|------------|------------------|--------------|
| **Logísticas** | `delivery_time_days`, `delay_vs_estimate`, `is_late` | Miden la eficiencia y puntualidad de las entregas |
| **Temporales** | `purchase_month` | Permite analizar estacionalidad |
| **Económicas** | `order_total_value` | Valor total de cada pedido |
| **Satisfacción** | `review_score` | Calificación otorgada por el cliente |

Este archivo será la base para el siguiente paso del proyecto, donde se exploran los datos en detalle para descubrir patrones, relaciones y posibles ideas sobre el comportamiento de las entregas y las valoraciones de los clientes.




### Principales hallazgos y siguientes pasos

Se creó un dataset enriquecido que combina tiempos de entrega, valor del pedido y calificaciones del cliente.

Observaciones clave: tiempo medio de entrega ≈ 5 días; calificación media ≈ 3.8; existen ~925 pedidos sin fecha de entrega (pendientes).

Se documentaron las decisiones de limpieza (conservación de NaT para entregas pendientes, conversión de fechas, agregación de valores por pedido).

Siguientes pasos: usar este dataset en el EDA (03_exploratory_analysis.ipynb) para explorar distribuciones, outliers y la relación entre logística y satisfacción.

### Exportación del dataset enriquecido

En la ejecución original del proyecto, el dataset enriquecido generado en este notebook fue almacenado como un archivo CSV para su reutilización en análisis posteriores.

Este archivo ya se encuentra versionado en el repositorio de GitHub dentro de la carpeta `data/processed`, por lo que el siguiente bloque de código se conserva únicamente como referencia del proceso realizado.


In [ ]:
#output_path = "/content/drive/MyDrive/ecommerce_project/data/processed/orders_enriched.csv"
#orders.to_csv(output_path, index=False)
#print(f"✅ Dataset enriquecido guardado en: {output_path}")
